In [5]:
import psutil
import time

# Memory Tracker

def get_current_process_memory_usage():
    process = psutil.Process()
    memory_info = process.memory_info()
    return memory_info.rss  # Returning Resident Set Size (physical memory usage)

def run_function_and_get_memory(target_function, num_runs=5, sleep_time=1):
    memory_usages = []

    for _ in range(num_runs):
        # Run the target function
        target_function()
        
        # Wait for a bit to ensure the process memory usage is stable
        time.sleep(sleep_time)
        
        # Get the memory usage of the current process
        memory_usage = get_current_process_memory_usage()
        
        # Add to the list if memory usage is found
        if memory_usage:
            memory_usages.append(memory_usage)
        
    if memory_usages:
        memory_usage = np.array(memory_usages)
        m = memory_usage.mean()
        std = memory_usage.std()
        return m, std
    else:
        return None

In [2]:
import unittest
import numpy as np
import pandas as pd
import ctypes
from customProphet import *

class TestLogPosteriorAndGradient(unittest.TestCase):
    def setUp(self):
        # Load the shared library
        self.lib = ctypes.CDLL('./libminus_log_posterior_and_gradient.so')
        self.lib.minus_log_posterior_and_gradient.argtypes = [
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
            ctypes.c_size_t,
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
            ctypes.c_size_t,
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
            ctypes.c_size_t,
            ctypes.c_double,
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
            ctypes.c_size_t,
            ctypes.c_double,
            ctypes.c_double,
            ctypes.c_double,
            ctypes.c_double,
            ctypes.c_double,
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS')
        ]
        self.lib.minus_log_posterior_and_gradient.restype = None

        # Load data
        df = pd.read_csv('peyton_manning.csv')

        # Instantiate & initialize a model
        self.model = CustomProphet()
        self.model.y = df['y'].values
        if df['ds'].dtype != 'datetime64[ns]':
            self.model.ds = pd.to_datetime(df['ds'])
        else:
            self.model.ds = df['ds']

        self.model.t_scaled = np.array((self.model.ds - self.model.ds.min()) / (self.model.ds.max() - self.model.ds.min()))
        self.model.T = df.shape[0]

        self.model.scale_period = (self.model.ds.max() - self.model.ds.min()).days
        self.model._normalize_y()
        self.model._generate_change_points()

        self.params = np.ones((47,))
    
    def tearDown(self):
        # This method is called after each test
        print('Test passed: combined function correctly implemented')

    def test_minus_log_posterior_and_gradient(self):
        # Convert data to ctypes
        params_ctypes = np.ascontiguousarray(self.params, dtype=np.float64)
        t_scaled_ctypes = np.ascontiguousarray(self.model.t_scaled, dtype=np.float64)
        change_points_ctypes = np.ascontiguousarray(self.model.change_points, dtype=np.float64)
        normalized_y_ctypes = np.ascontiguousarray(self.model.normalized_y, dtype=np.float64)

        mlp_out = np.zeros((1,), dtype=np.float64)
        grad_out = np.zeros((47,), dtype=np.float64)

        # Call the Python method
        mpl_python, grad_python = self.model._minus_log_posteriorAndGradient(self.params)

        # Call the C++ function
        self.lib.minus_log_posterior_and_gradient(
            params_ctypes, len(params_ctypes),
            t_scaled_ctypes, len(t_scaled_ctypes),
            change_points_ctypes, len(change_points_ctypes),
            self.model.scale_period,
            normalized_y_ctypes, len(normalized_y_ctypes),
            self.model.sigma_obs,
            self.model.sigma_k,
            self.model.sigma_m,
            self.model.sigma,
            self.model.tau,
            mlp_out,
            grad_out
        )

        # Check if outputs match
        np.testing.assert_almost_equal(mlp_out[0], mpl_python, decimal=5, err_msg="Log-posterior mismatch")
        np.testing.assert_almost_equal(grad_out, grad_python, decimal=5, err_msg="Gradient mismatch")
# Run in a jupyter notebook
suite = unittest.TestLoader().loadTestsFromTestCase(TestLogPosteriorAndGradient)

# Run the test suite
unittest.TextTestRunner(verbosity=2).run(suite)

#if __name__ == '__main__':
#    unittest.main()

test_minus_log_posterior_and_gradient (__main__.TestLogPosteriorAndGradient.test_minus_log_posterior_and_gradient) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.012s

OK


Test passed: combined function correctly implemented


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [3]:
import numpy as np
import pandas as pd
import ctypes
from customProphet import *

# Load the shared library
lib = ctypes.CDLL('./libminus_log_posterior_and_gradient.so')

# Define argument and return types for the C++ function
lib.minus_log_posterior_and_gradient.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS')
]
lib.minus_log_posterior_and_gradient.restype = None

# Load data
df = pd.read_csv('peyton_manning.csv')

# Instantiate & initialize a model
model = CustomProphet()
model.y = df['y'].values
if df['ds'].dtype != 'datetime64[ns]':
    model.ds = pd.to_datetime(df['ds'])
else:
    model.ds = df['ds']

model.t_scaled = np.array((model.ds - model.ds.min()) / (model.ds.max() - model.ds.min()))
model.T = df.shape[0]

model.scale_period = (model.ds.max() - model.ds.min()).days
model._normalize_y()
model._generate_change_points()

params = np.ones((47,))

# Convert data to ctypes
params_ctypes = np.ascontiguousarray(params, dtype=np.float64)
t_scaled_ctypes = np.ascontiguousarray(model.t_scaled, dtype=np.float64)
change_points_ctypes = np.ascontiguousarray(model.change_points, dtype=np.float64)
normalized_y_ctypes = np.ascontiguousarray(model.normalized_y, dtype=np.float64)

# Call the Python method
python_loss, python_grad = model._minus_log_posteriorAndGradient(params)

# Prepare arrays to hold C++ results
cpp_loss = np.zeros((1,), dtype=np.float64)
cpp_grad = np.zeros((47,), dtype=np.float64)

# Call the C++ function
lib.minus_log_posterior_and_gradient(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau,
    cpp_loss,
    cpp_grad
)

# Print the results
print("Python loss:", python_loss)
print("Python gradient:", python_grad)
print("C++ loss:", cpp_loss[0])
print("C++ gradient:", cpp_grad)

Python loss: 3594756527.386767
Python gradient: [4.53910463e+08 6.27722325e+08 4.34082096e+08 4.14319290e+08
 3.94554629e+08 3.74729689e+08 3.55168271e+08 3.35701626e+08
 3.16288040e+08 2.96918974e+08 2.77885560e+08 2.59034011e+08
 2.40338494e+08 2.21843054e+08 2.03823971e+08 1.86141436e+08
 1.68782897e+08 1.51841098e+08 1.35530190e+08 1.19756852e+08
 1.04529979e+08 9.00287665e+07 7.64151900e+07 6.36126049e+07
 5.16235480e+07 4.07242028e+07 3.09840090e+07 7.98137483e+07
 7.77656086e+07 6.96585489e+07 6.23789275e+07 5.45990584e+07
 5.24717230e+07 5.08324128e+07 5.27016116e+07 5.65869075e+07
 5.64630459e+07 3.02496570e+07 5.08828269e+07 6.02259051e+07
 6.40012391e+07 6.36707278e+07 5.99211057e+07 5.63612029e+07
 5.47744602e+07 5.35111771e+07 5.50934614e+07]
C++ loss: 3594756527.386767
C++ gradient: [4.53910463e+08 6.27722325e+08 4.34082096e+08 4.14319290e+08
 3.94554629e+08 3.74729689e+08 3.55168271e+08 3.35701626e+08
 3.16288040e+08 2.96918974e+08 2.77885560e+08 2.59034011e+08
 2.403384

In [4]:
%%timeit -r 100 -n 100

lib.minus_log_posterior_and_gradient(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau,
    cpp_loss,
    cpp_grad
)

434 µs ± 41.5 µs per loop (mean ± std. dev. of 100 runs, 100 loops each)


In [6]:
def run():
    lib.minus_log_posterior_and_gradient(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau,
    cpp_loss,
    cpp_grad
    )

average_memory_usage, std = run_function_and_get_memory(run, num_runs=100, sleep_time=1)
if average_memory_usage:
    print(f"Average memory usage over 30 runs: {average_memory_usage / 1024**2} MB +/- {std / 1024**2} MB")
else:
    print("Failed to measure memory usage.")

Average memory usage over 30 runs: 66.22921875 MB +/- 6.919423611842044 MB


In [5]:
%%timeit -r 100 -n 100

lib.minus_log_posterior_and_gradient(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau,
    cpp_loss,
    cpp_grad
)

427 µs ± 20.5 µs per loop (mean ± std. dev. of 100 runs, 100 loops each)


In [8]:
%%timeit -r 100 -n 100

python_loss, python_grad = model._minus_log_posteriorAndGradient(params)

1.43 ms ± 361 µs per loop (mean ± std. dev. of 100 runs, 100 loops each)


In [8]:
def run():
    model._minus_log_posteriorAndGradient(params)
    
average_memory_usage, std = run_function_and_get_memory(run, num_runs=100, sleep_time=1)
if average_memory_usage:
    print(f"Average memory usage over 30 runs: {average_memory_usage / 1024**2} MB +/- {std / 1024**2} MB")
else:
    print("Failed to measure memory usage.")

Average memory usage over 30 runs: 59.34109375 MB +/- 7.346520107535927 MB


In [1]:
import unittest
import numpy as np
import pandas as pd
import ctypes
from customProphet import *

class TestLogPosteriorAndGradient(unittest.TestCase):
    def setUp(self):
        # Load the shared library
        self.lib = ctypes.CDLL('./libminus_log_posterior_and_gradient_2.so')
        self.lib.minus_log_posterior_and_gradient.argtypes = [
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
            ctypes.c_size_t,
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
            ctypes.c_size_t,
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
            ctypes.c_size_t,
            ctypes.c_double,
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
            ctypes.c_size_t,
            ctypes.c_double,
            ctypes.c_double,
            ctypes.c_double,
            ctypes.c_double,
            ctypes.c_double,
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
            np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS')
        ]
        self.lib.minus_log_posterior_and_gradient.restype = None

        # Load data
        df = pd.read_csv('peyton_manning.csv')

        # Instantiate & initialize a model
        self.model = CustomProphet()
        self.model.y = df['y'].values
        if df['ds'].dtype != 'datetime64[ns]':
            self.model.ds = pd.to_datetime(df['ds'])
        else:
            self.model.ds = df['ds']

        self.model.t_scaled = np.array((self.model.ds - self.model.ds.min()) / (self.model.ds.max() - self.model.ds.min()))
        self.model.T = df.shape[0]

        self.model.scale_period = (self.model.ds.max() - self.model.ds.min()).days
        self.model._normalize_y()
        self.model._generate_change_points()

        self.params = np.ones((47,))
    
    def tearDown(self):
        # This method is called after each test
        print('Test passed: combined function correctly implemented')

    def test_minus_log_posterior_and_gradient(self):
        # Convert data to ctypes
        params_ctypes = np.ascontiguousarray(self.params, dtype=np.float64)
        t_scaled_ctypes = np.ascontiguousarray(self.model.t_scaled, dtype=np.float64)
        change_points_ctypes = np.ascontiguousarray(self.model.change_points, dtype=np.float64)
        normalized_y_ctypes = np.ascontiguousarray(self.model.normalized_y, dtype=np.float64)

        mlp_out = np.zeros((1,), dtype=np.float64)
        grad_out = np.zeros((47,), dtype=np.float64)

        # Call the Python method
        mpl_python, grad_python = self.model._minus_log_posteriorAndGradient(self.params)

        # Call the C++ function
        self.lib.minus_log_posterior_and_gradient(
            params_ctypes, len(params_ctypes),
            t_scaled_ctypes, len(t_scaled_ctypes),
            change_points_ctypes, len(change_points_ctypes),
            self.model.scale_period,
            normalized_y_ctypes, len(normalized_y_ctypes),
            self.model.sigma_obs,
            self.model.sigma_k,
            self.model.sigma_m,
            self.model.sigma,
            self.model.tau,
            mlp_out,
            grad_out
        )

        # Check if outputs match
        np.testing.assert_almost_equal(mlp_out[0], mpl_python, decimal=5, err_msg="Log-posterior mismatch")
        np.testing.assert_almost_equal(grad_out, grad_python, decimal=5, err_msg="Gradient mismatch")
# Run in a jupyter notebook
suite = unittest.TestLoader().loadTestsFromTestCase(TestLogPosteriorAndGradient)

# Run the test suite
unittest.TextTestRunner(verbosity=2).run(suite)

#if __name__ == '__main__':
#    unittest.main()

test_minus_log_posterior_and_gradient (__main__.TestLogPosteriorAndGradient.test_minus_log_posterior_and_gradient) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.062s

OK


Test passed: combined function correctly implemented


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [2]:
import numpy as np
import pandas as pd
import ctypes
from customProphet import *

# Load the shared library
lib = ctypes.CDLL('./libminus_log_posterior_and_gradient_2.so')

# Define argument and return types for the C++ function
lib.minus_log_posterior_and_gradient.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS')
]
lib.minus_log_posterior_and_gradient.restype = None

# Load data
df = pd.read_csv('peyton_manning.csv')

# Instantiate & initialize a model
model = CustomProphet()
model.y = df['y'].values
if df['ds'].dtype != 'datetime64[ns]':
    model.ds = pd.to_datetime(df['ds'])
else:
    model.ds = df['ds']

model.t_scaled = np.array((model.ds - model.ds.min()) / (model.ds.max() - model.ds.min()))
model.T = df.shape[0]

model.scale_period = (model.ds.max() - model.ds.min()).days
model._normalize_y()
model._generate_change_points()

params = np.ones((47,))

# Convert data to ctypes
params_ctypes = np.ascontiguousarray(params, dtype=np.float64)
t_scaled_ctypes = np.ascontiguousarray(model.t_scaled, dtype=np.float64)
change_points_ctypes = np.ascontiguousarray(model.change_points, dtype=np.float64)
normalized_y_ctypes = np.ascontiguousarray(model.normalized_y, dtype=np.float64)

# Call the Python method
python_loss, python_grad = model._minus_log_posteriorAndGradient(params)

# Prepare arrays to hold C++ results
cpp_loss = np.zeros((1,), dtype=np.float64)
cpp_grad = np.zeros((47,), dtype=np.float64)

# Call the C++ function
lib.minus_log_posterior_and_gradient(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau,
    cpp_loss,
    cpp_grad
)

# Print the results
print("Python loss:", python_loss)
print("Python gradient:", python_grad)
print("C++ loss:", cpp_loss[0])
print("C++ gradient:", cpp_grad)

Python loss: 19106.191245889222
Python gradient: [2349.42929854 3249.05986776 2266.75980306 2164.46975155 2062.17009766
 1959.55844878 1858.3107567  1757.55360413 1657.07107316 1556.81897732
 1458.30417478 1360.73068588 1263.96480128 1168.23448928 1074.96975169
  983.44694692  893.60111154  805.91227332  721.48885286  639.84783933
  561.03526554  485.978625    415.5162905   349.25155575  287.19755094
  230.7837715   180.36962795  413.11694771  402.51600831  360.55478261
  322.87628419  282.60856058  271.59771251  263.11282792  272.78759014
  292.89744283  292.2563492   156.5788083   263.37376582  311.73248328
  331.27318593  329.56249673  310.15487623  291.72922157  283.51642046
  276.97780949  285.16753446]
C++ loss: 19106.191245889222
C++ gradient: [2349.42929854 3249.05986776 2266.75980306 2164.46975155 2062.17009766
 1959.55844878 1858.3107567  1757.55360413 1657.07107316 1556.81897732
 1458.30417478 1360.73068588 1263.96480128 1168.23448928 1074.96975169
  983.44694692  893.601111

In [3]:
%%timeit -r 100 -n 100

lib.minus_log_posterior_and_gradient(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau,
    cpp_loss,
    cpp_grad
)

413 µs ± 21.8 µs per loop (mean ± std. dev. of 100 runs, 100 loops each)


***
## Memory tracking

In [6]:
def run():
    lib.minus_log_posterior_and_gradient(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau,
    cpp_loss,
    cpp_grad
    )

average_memory_usage, std = run_function_and_get_memory(run, num_runs=100, sleep_time=1)
if average_memory_usage:
    print(f"Average memory usage over 30 runs: {average_memory_usage / 1024**2} MB +/- {std / 1024**2} MB")
else:
    print("Failed to measure memory usage.")

Average memory usage over 30 runs: 63.14453125 MB +/- 8.962496560919407 MB


In [ ]:
import numpy as np
import pandas as pd
import ctypes
from customProphet import *

# Load the shared library
lib = ctypes.CDLL('./libminus_log_posterior_and_gradient.so')

# Define argument and return types for the C++ function
lib.minus_log_posterior_and_gradient.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS')
]
lib.minus_log_posterior_and_gradient.restype = None

# Load data
df = pd.read_csv('peyton_manning.csv')

# Instantiate & initialize a model
model = CustomProphet()
model.y = df['y'].values
if df['ds'].dtype != 'datetime64[ns]':
    model.ds = pd.to_datetime(df['ds'])
else:
    model.ds = df['ds']

model.t_scaled = np.array((model.ds - model.ds.min()) / (model.ds.max() - model.ds.min()))
model.T = df.shape[0]

model.scale_period = (model.ds.max() - model.ds.min()).days
model._normalize_y()
model._generate_change_points()

params = np.ones((47,))

# Convert data to ctypes
params_ctypes = np.ascontiguousarray(params, dtype=np.float64)
t_scaled_ctypes = np.ascontiguousarray(model.t_scaled, dtype=np.float64)
change_points_ctypes = np.ascontiguousarray(model.change_points, dtype=np.float64)
normalized_y_ctypes = np.ascontiguousarray(model.normalized_y, dtype=np.float64)

# Call the Python method
python_loss, python_grad = model._minus_log_posteriorAndGradient(params)

# Prepare arrays to hold C++ results
cpp_loss = np.zeros((1,), dtype=np.float64)
cpp_grad = np.zeros((47,), dtype=np.float64)

# Call the C++ function
lib.minus_log_posterior_and_gradient(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau,
    cpp_loss,
    cpp_grad
)

def wrapper(params):
    